# Описание задачи


Многие знают про маркетплейсы где продаются б/у вещи, на которых есть возможность недорого купить качественную и полезную вещь. Но всегда волнует вопрос - кто и как устанавливает цену, и какие его характеристики больше всего влияют на итоговую стоимость продажи?! Вопрос становиться особо актуальным, если речь идет про дорогие товары, например про автомобили!    

В рамках данного исследования будет вестить работа с данными о продажах автомобилей на вторичном рынке. 

**Целью проекта будет *разработанная модель* предсказания стоимости автомобиля на вторичном рынке.**


## Данные
**train.csv** - информация о продажах (~440000) автомобилей с аукционов, которые будут использоваться в качестве обучающих данных.

**test.csv** - информация о продажах (~110000) автомобилей с аукционов, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'sellingprice' для каждого автомобиля из этого датасета.

**sample_submission.csv** - файл предсказаний в правильном формате.
vin - идентификатор каждого автомобиля в тестовом наборе.
sellingprice - Целевой признак. Для каждого автомобиля предскажите числовое значение стоимости автомобиля.

## Описание полей данных
`year` - год производства

`make` - производитель

`model` - модель

`trim` - модификация

`body` - тип кузова

`transmission` - тип КПП

`vin` - идентификатор (вин)

`state` - штат регистрации

`condition` - состояние по шкале (1-5)

`odometer` - пробег в милях

`color` - цвет кузова

`interior` - цвет интерьера

`seller` - продавец

`sellingprice` - стоимость продажи

`saledate` - дата продажи

## Ход решения задачи

Пошагово:

1. загрузка и ознакомление с данными,
2. предварительная обработка,
3. полноценный разведочный анализ,
4. разработка новых синтетических признаков,
5. проверка на мультиколлинеарность,
6. отбор финального набора обучающих признаков,
7. выбор и обучение моделей,
8. итоговая оценка качества предсказания лучшей модели,
9. анализ важности ее признаков.

### 1. Загрузка и ознакомление с данными

In [1]:
!pip install category_encoders -U

In [2]:
!pip install catboost -U

In [3]:
#импортируем необходимые для работы библиотеки
import pandas as pd
pd.set_option
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import OrdinalEncoder

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer

from sklearn.base import TransformerMixin

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_percentage_error as mape

После загрузки необходимых библиотек подгрузим датасеты

In [4]:
df_train = pd.read_csv('/Users/moygospodin/Documents/Practicum/Projects/Automobile_research/datasets/train.csv')
df_test = pd.read_csv('/Users/moygospodin/Documents/Practicum/Projects/Automobile_research/datasets/test.csv')

Проверим, что все отображается верно

In [5]:
display(df_train.head(5))
display(df_test.head(5))

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
0,2005,Cadillac,CTS,Base,Sedan,automatic,1g6dp567450124779,ca,2.7,116970.0,silver,black,lexus of stevens creek,Wed Jan 14 2015 04:30:00 GMT-0800 (PST)
1,2014,GMC,Savana Cargo,2500,Van,NaN,1gtw7fca7e1902207,pa,4.4,6286.0,white,gray,u-haul,Fri Feb 27 2015 01:00:00 GMT-0800 (PST)
2,2013,Nissan,Murano,S,SUV,automatic,jn8az1mw6dw303497,oh,4.6,11831.0,gray,black,nissan-infiniti lt,Tue Feb 24 2015 01:30:00 GMT-0800 (PST)
3,2013,Chevrolet,Impala,LS Fleet,Sedan,automatic,2g1wf5e34d1160703,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,Fri Mar 06 2015 02:00:00 GMT-0800 (PST)
4,2013,Nissan,Titan,SV,Crew Cab,automatic,1n6aa0ec3dn301209,tn,2.9,31083.0,black,black,nissan north america inc.,Wed Jun 03 2015 03:30:00 GMT-0700 (PDT)


### 2. Предобработка данных

Нужно посмотреть, все ли данные корректны в датасетах. Удалить дубликаты, если таковые имеются и убрать столбцы с данными, которые либо дублируют друг друга, либо не помогут в выполнении задачи. Проверить на выбросы и пропуски в данных.


In [6]:
for i in [df_train, df_test]:
    display(i.info())
    display(i.isna().sum())
    display(i.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


None

year                0
make             8043
model            8123
trim             8337
body            10393
transmission    51461
vin                 0
state               0
condition        9405
odometer           69
color             586
interior          586
seller              0
sellingprice        0
saledate            0
dtype: int64

0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 entries, 0 to 110059
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110060 non-null  int64  
 1   make          107999 non-null  object 
 2   model         107981 non-null  object 
 3   trim          107946 non-null  object 
 4   body          107466 non-null  object 
 5   transmission  97048 non-null   object 
 6   vin           110060 non-null  object 
 7   state         110060 non-null  object 
 8   condition     107681 non-null  float64
 9   odometer      110041 non-null  float64
 10  color         109902 non-null  object 
 11  interior      109902 non-null  object 
 12  seller        110060 non-null  object 
 13  saledate      110060 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


None

year                0
make             2061
model            2079
trim             2114
body             2594
transmission    13012
vin                 0
state               0
condition        2379
odometer           19
color             158
interior          158
seller              0
saledate            0
dtype: int64

0

везде нижний регистр

In [7]:
df_train['saledate'] = pd.to_datetime(df_train['saledate'], utc=True)

In [8]:
df_test['saledate'] = pd.to_datetime(df_test['saledate'], utc=True)

In [9]:
df_train = df_train.applymap(lambda s: s.lower() if type(s) == str else s)
df_test = df_test.applymap(lambda s: s.lower() if type(s) == str else s)

In [10]:
for i in df_train.columns:
    display(df_train[i].value_counts())

2012    80625
2013    77566
2014    64225
2011    38115
2008    24646
2007    24152
2006    21118
2010    20647
2005    16794
2009    16064
2004    13658
2003    10512
2002     7726
2015     7462
2001     5147
2000     4161
1999     2658
1998     1747
1997     1224
1996      683
1995      567
1994      319
1993      168
1992      103
1991       56
1990       38
1989       15
1988       11
1985       10
1986        8
1987        5
1984        4
1983        1
1982        1
Name: year, dtype: int64

ford         74067
chevrolet    47818
nissan       42866
toyota       31711
dodge        24294
             ...  
daewoo           1
dodge tk         1
mazda tk         1
dot              1
airstream        1
Name: make, Length: 62, dtype: int64

altima            15454
f-150             11408
fusion            10244
camry             10000
escape             9447
                  ...  
420-class             1
c230                  1
rrs                   1
activehybrid 5        1
g500                  1
Name: model, Length: 839, dtype: int64

base                43876
se                  34498
lx                  16511
limited             14516
lt                  13431
                    ...  
cl65 amg                1
cargo awd w/yf7         1
gr tr gr touring        1
sub 4x4 v8              1
4x4 v6 xlt sport        1
Name: trim, Length: 1850, dtype: int64

sedan                      190861
suv                        113042
hatchback                   20715
minivan                     20114
coupe                       13896
crew cab                    12823
wagon                       12600
convertible                  8183
supercrew                    7081
g sedan                      5859
supercab                     4143
regular cab                  3861
van                          3544
extended cab                 3500
quad cab                     3208
e-series van                 1426
g coupe                      1282
double cab                   1260
crewmax cab                   429
king cab                      418
g convertible                 251
access cab                    232
genesis coupe                 221
club cab                      141
koup                          135
cts coupe                     115
mega cab                       94
elantra coupe                  74
beetle convertible             52
promaster carg

...

black        87115
white        84149
silver       65667
gray         65284
blue         40237
red          34514
—            19520
green         8975
gold          8934
beige         7257
burgundy      7059
brown         5320
orange        1629
purple        1250
off-white     1143
yellow         979
charcoal       389
turquoise      183
pink            32
lime            14
Name: color, dtype: int64

black        192442
gray         140843
beige         46878
tan           34709
—             13563
brown          6818
red            1070
blue            885
silver          844
off-white       373
purple          276
gold            256
white           215
green           198
burgundy        155
orange          109
yellow           16
Name: interior, dtype: int64

nissan-infiniti lt                15823
ford motor credit company,llc     15276
the hertz corporation             14692
santander consumer                12230
avis corporation                  10098
                                  ...  
bay shore motors                      1
thrifty rent a car systems inc        1
volvo of wichita                      1
wright way hyundai                    1
studio city auto group                1
Name: seller, Length: 13025, dtype: int64

11000    3510
12000    3499
13000    3374
10000    3178
14000    3104
         ... 
57100       1
73100       1
41501       1
66700       1
2725        1
Name: sellingprice, Length: 1735, dtype: int64

2015-02-09 17:30:00+00:00    4209
2015-01-26 17:30:00+00:00    3878
2015-02-16 17:30:00+00:00    3862
2015-01-19 17:30:00+00:00    3777
2015-03-02 17:30:00+00:00    3633
                             ... 
2015-01-06 21:10:00+00:00       1
2015-06-24 20:35:00+00:00       1
2015-06-16 23:15:00+00:00       1
2015-04-01 02:31:00+00:00       1
2015-05-19 18:30:00+00:00       1
Name: saledate, Length: 3597, dtype: int64

Заполняем пропуски

In [11]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [12]:
df_train_fill = DataFrameImputer().fit_transform(df_train)

In [13]:
df_test_fill = DataFrameImputer().fit_transform(df_test)

In [14]:
df_train_fill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   year          440236 non-null  int64              
 1   make          440236 non-null  object             
 2   model         440236 non-null  object             
 3   trim          440236 non-null  object             
 4   body          440236 non-null  object             
 5   transmission  440236 non-null  object             
 6   vin           440236 non-null  object             
 7   state         440236 non-null  object             
 8   condition     440236 non-null  float64            
 9   odometer      440236 non-null  float64            
 10  color         440236 non-null  object             
 11  interior      440236 non-null  object             
 12  seller        440236 non-null  object             
 13  sellingprice  440236 non-null  int64        

In [17]:
%%time
#тут работает энкодер
#data = pd.DataFrame(OrdinalEncoder().fit_transform(df_train_fill),
#                            columns=df_train_fill.columns)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.78 µs


In [18]:
%%time
#тут работает энкодер
#data_test = pd.DataFrame(OrdinalEncoder().fit_transform(df_test_fill),
#                            columns=df_test_fill.columns)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


In [19]:
#data.info()

In [20]:
features_train = df_train_fill.drop(['sellingprice'], axis=1)
target_train = df_train_fill['sellingprice']

features_test = df_test_fill


#features_train = data.drop(['sellingprice'], axis=1)
#target_train = data['sellingprice']

#features_test = data_test


In [27]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   year          440236 non-null  int64              
 1   make          440236 non-null  object             
 2   model         440236 non-null  object             
 3   trim          440236 non-null  object             
 4   body          440236 non-null  object             
 5   transmission  440236 non-null  object             
 6   vin           440236 non-null  object             
 7   state         440236 non-null  object             
 8   condition     440236 non-null  float64            
 9   odometer      440236 non-null  float64            
 10  color         440236 non-null  object             
 11  interior      440236 non-null  object             
 12  seller        440236 non-null  object             
 13  saledate      440236 non-null  datetime64[ns

In [30]:
cat = CatBoostRegressor(cat_features = [1,2,3,4,5,6,7,10,11,12,13],
                        loss_function = "MAPE",
                       eval_metric = "MAPE")
cat_features = [1,2,3,4,5,6,7,10,11,12,13]

In [31]:
%%time

#это поиск параметров

param_grid = {   
          'depth':[5, 8, 10], #[3,1,2,6,4,5,7,8,9,10],
          'iterations':[50, 100], #[250,100,500,1000],
          'learning_rate':[0.01, 0.3], #[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[5, 10, 20], #[3,1,5,10,100],
}
gs_result = cat.grid_search(param_grid,
                            partition_random_seed=1602,
                            X=features_train,
                            y=target_train
)

TypeError: must be real number, not Timestamp

In [ ]:
cat.get_params()

In [ ]:
%%time
#это кросс-валидация

params = {
    'loss_function': 'MAPE',
  'depth': 10,
 'l2_leaf_reg': 20,
 'iterations': 100,
 'learning_rate': 0.3
}
cv_dataset = Pool(data=features_train,
                  label=target_train)
scores = cv(cv_dataset,
            params,
            fold_count=2)

In [ ]:
predict_data = cat.predict(features_test)

In [ ]:
df_test_fill.info()

In [ ]:
submission = df_test_fill['vin']

In [ ]:
predict_series = pd.Series(predict_data)

In [ ]:
submission2 = submission.to_frame()

In [ ]:
submission2['sellingprice'] = predict_series

In [ ]:
#submission2.to_csv('sub1.csv', index=False)

In [ ]:
submission2.head()